In [205]:
import requests
import json
# import openai
from openai import AzureOpenAI
import time
from dotenv import load_dotenv
import os, time, glob
from tqdm import tqdm
from statistics import mode
from sklearn.metrics import confusion_matrix
import pandas as pd

load_dotenv()

True

In [121]:
os.getenv("GPT-4O")

'GPT-4O'

In [192]:
test = """you might want to check dosing, you may be on too high of a dose or vice versa im only sleepy when I'm on too high of a dose and I'm 4 years in"""

In [99]:
context = """You are an AI assistant that helps to annotate social media comment data related to opioids and/or drug addiction, and you know everything about identifying the grammartical person (ie first, second, or third person) of a sentence or post"""

In [218]:
prompt = """I will give you a social media comment to annotate. This comment is part of a larger drugs (such as opioids, etc) discussion, and you will tell me what the person's perpective in relation to drug is. This relation can take many forms. It could be active use of drugs (such as opioids), recovery from addiction, loss of a loved one to drugs, etc. You will return a onehot integer of length 3 (eg. 010). 

If the comment contains a personal account with opioids put 1 in position 1 of the onehot integer else 0.

If the comment contains another person's experience or account with opioid, put 1 in position 2 of the onehot integer else 0. 

If the comment contains a third-person experience with opioid, put 1 in position 3 of the onehot integer else 0. 

Examples: return 100 for 'I am recovering from OUD', 010 for 'I am sorry to hear you lost your son to fentanyl OD', 001 for 'They take fent laced substances'. 

Note that some comments can have multiple persons-perspective and you will put 1 in the positions corresponding to the persons-perspective present. Example return 110 for 'I lost a partner. I am sorry to hear you lost someone too. My condolences to you and your family', and 011 for 'You should take lower dose. They should be careful with supplies from the street'.

Also note that you are making this decision based on only opioid aspects of the comments I give you.
  
Consider the following comment, reason about it for a bit and then the annotation and explain:\n""" + test + "\n\nResponse format: [Annotation]\n[Explanation]"

In [208]:
print(prompt)

I will give you a social media comment to annotate. This comment is part of a larger opioid discussion, and you will tell me what the person's perpective in relation to opioid is. This relation can take many forms. It could be active use of opioids or drugs, recovery from addiction, loss of a loved one to opioids or drugs, etc. You will return a onehot integer of length 3 (eg. 010). 

If the comment contains a personal account with opioids put 1 in position 1 of the onehot integer else 0.

If the comment contains another person's experience or account with opioid, put 1 in position 2 of the onehot integer else 0. 

If the comment contains a third-person experience with opioid, put 1 in position 3 of the onehot integer else 0. 

Examples: return 100 for 'I am recovering from OUD', 010 for 'I am sorry to hear you lost your son to fentanyl OD', 001 for 'They take fent laced substances'. 

Note that some comments can have multiple persons-perspective and you will put 1 in the positions cor

In [195]:
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-01"
)

response = client.chat.completions.create(
    model=os.getenv("GPT-4O"), # model = "deployment_name".
    messages=[
        {"role": "system", "content": context},
        {"role": "user", "content": prompt},
    ]
)

print(response.choices[0].message.content)

Given the comment: "you might want to check dosing, you may be on too high of a dose or vice versa im only sleepy when I'm on too high of a dose and I'm 4 years in".

Let's break it down:

1. The person starts with a suggestion to another person: "you might want to check dosing, you may be on too high of a dose or vice versa." This indicates they are addressing someone else and discussing that person's opioid dosing.

2. Then they mention their own experience: "im only sleepy when I'm on too high of a dose and I'm 4 years in," which indicates past or ongoing personal use, emphasizing their experience over four years.

These perspectives map directly onto the given conditions:

- The first part of the comment includes a second-person perspective, addressing 'you':
  - Thus, we will set position 2 (second person) to 1.
- The second part of the comment includes a first-person perspective, reflecting on the speaker's own experience:
  - Thus, we will set position 1 (first person) to 1.

Si

In [128]:
endpoint_url = "https://tiktokplayground.openai.azure.com/openai/deployments/o1-mini-2/chat/completions?api-version=2024-08-01-preview"
key = "coiL9LJyH2NWxiTK49ubEi0ulIJu4ECWAPTlbVbWEDmoLRRx8croJQQJ99BBACYeBjFXJ3w3AAABACOGtsy2"
o1_mini = "o1-mini-2"

In [196]:
client = AzureOpenAI(
  azure_endpoint = endpoint_url, 
  api_key=key,  
  api_version="2024-02-01"
)

response = client.chat.completions.create(
    model=o1_mini, # model = "deployment_name".
    messages=[
        {"role": "user", "content": prompt},
    ]
)

print(response.choices[0].message.content)

**Annotation:** `110`

**Explanation:**
- **Position 1 (Personal account):** The commenter shares their own experience with dosing and being on opioids for four years ("I'm only sleepy when I'm on too high of a dose and I'm 4 years in"), indicating a personal involvement.
- **Position 2 (Another person's experience):** They address someone else directly by offering advice ("you might want to check dosing, you may be on too high of a dose or vice versa"), which relates to another person's experience.
- **Position 3 (Third-person experience):** There's no mention of a third person's experience in the comment.

Therefore, the one-hot annotation is `110`.


In [203]:
def query_o1(prompt):
    client = AzureOpenAI(
      azure_endpoint = os.getenv("o1_endpoint"), 
      api_key=os.getenv("o1_key"),  
      api_version="2024-02-01"
    )
    
    response = client.chat.completions.create(
        model=os.getenv("o1_mini"), # model = "deployment_name".
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    return response.choices[0].message.content

In [212]:
print(query_o1(prompt))

100
The comment includes a personal account of opioid use, as the individual mentions their own experience with dosing ("I'm only sleepy when I'm on too high of a dose") and the duration of their use ("I'm 4 years in"). There are no references to another person's or a third person's experiences.


In [73]:
def query(context, prompt):
    client = AzureOpenAI(
      azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
      api_key=os.getenv("AZURE_OPENAI_KEY"),  
      api_version="2024-02-01"
    )
    
    response = client.chat.completions.create(
        model=os.getenv("DEPLOYMENT_NAME"), # model = "deployment_name".
        messages=[
            {"role": "system", "content": context},
            {"role": "user", "content": prompt},
        ]
    )
    return response.choices[0].message.content

# Person - o1_mini

In [223]:
prompt_template = """I will give you a social media comment to annotate. This comment is part of a larger drugs (such as opioids, etc) discussion, and you will tell me what the person's perpective in relation to drug is. This relation can take many forms. It could be active use of drugs, recovery from addiction, loss of a loved one to drugs, etc. You will return a onehot integer of length 3 (eg. 010). 

If the comment contains a personal account with drugs put 1 in position 1 of the onehot integer else 0.

If the comment contains another person's experience or account with drugs, put 1 in position 2 of the onehot integer else 0. 

If the comment contains a third-person experience with drugs, put 1 in position 3 of the onehot integer else 0. 

Examples: return 100 for 'I am recovering from OUD', 010 for 'I am sorry to hear you lost your son to fentanyl OD', 001 for 'They take fent laced substances'. 

Note that some comments can have multiple persons-perspective and you will put 1 in the positions corresponding to the persons-perspective present. Example return 110 for 'I lost a partner. I am sorry to hear you lost someone too. My condolences to you and your family', and 011 for 'You should take lower dose. They should be careful with supplies from the street'.

Also note that you are making this decision based on only drug aspects of the comments I give you.
  
Consider the following comment, reason about it for a bit and then the annotation and explain:\n"""

In [224]:
format = "\n\nResponse format: [Annotation]\n[Explanation]"

In [225]:
with open("./persons.txt") as f:
    comments = f.read()
    comments = comments.split("\n\n")

In [ ]:
for i in tqdm(range(len(comments))):
    comment = comments[i]
    text = ''
    for _ in range(20):
        prompt = prompt_template + comment + format
        response = query_o1(prompt)
        annot = response.split("\n")[0]
        text += annot + "\n"
        time.sleep(0.5)
    with open(f"./persons_predictions/sample_{i}.txt", "w") as f:
        f.write(text)

 11%|█         | 3/28 [06:00<50:41, 121.68s/it]

# Persons

In [49]:
context = """You are an AI assistant that helps to annotate social media comment data related to opioids and/or drug addiction, and you know everything about identifying the grammartical person (ie first, second, or third person) of a sentence or post"""

In [50]:
prompt_template = """I will give you an opioid-related social media comment to annotate. You will return a onehot integer of length 3 (eg. 010). If the comment contains a first-person or personal account with opioids put 1 in position 1 of the onehot integer else 0. If the comment contains a second-person experience with opioid, put 1 in position 2 of the onehot integer else 0. If the comment contains a third-person experience with opioid, put 1 in position 3 of the onehot integer else 0. Examples: return 100 for 'I am recovering from OUD', 010 for 'I am sorry to hear you lost your son to fentanyl OD', 001 for 'They take fent laced substances'. Note that some comments can have multiple persons-perspective and you will put 1 in the positions corresponding to the persons-perspective present. Example return 110 for 'I lost a partner. I am sorry to hear you lost someone too. My condolences to you and your family', and 011 for 'You should take lower dose. They should be careful with supplies from the street'. 
Annotate the following (only return the annotation):\n"""

In [51]:
with open("./persons.txt") as f:
    comments = f.read()
    comments = comments.split("\n\n")

In [52]:
for i in tqdm(range(len(comments))):
    comment = comments[i]
    text = ''
    for _ in range(20):
        prompt = prompt_template + comment
        response = query(context, prompt)
        text += response + "\n"
        time.sleep(0.5)
    with open(f"./persons_predictions/sample_{i}.txt", "w") as f:
        f.write(text)

100%|██████████| 28/28 [18:52<00:00, 40.44s/it]


# Expert

In [42]:
context = "You are an AI assistant that helps to annotate social media comment data related to opioids and/or drug addiction, and you can differentiate between comments generated by experts and those that are not"

In [39]:
prompt_template = """I will give you an opioid-related social media comment to annotate. Return 1 if the post is written by an expert and 0 if it is not. An expert here is broadly defined as someone who works in healthcare. Some posts have hints that the author works in healthcare. Example: 'Neurosurgeon here, I see alot of patients', 'I am a physician assistant, and I have seen a lot of that', 'My patients respond well to XYZ drug'. Post that don't have hint that the author work in healthcare is not considered generated by an expert. Example: '8 months clean now', 'I am so sorry to hear this', etc. Return 1 for generated comments, 0 otherwise.
Annotate the following:\n"""

In [44]:
with open("./experts.txt") as f:
    comments = f.read()
    comments = comments.split("\n\n")

In [48]:
text = ""
for i in tqdm(range(len(comments))):
    comment = comments[i]
    prompt = prompt_template + comment
    response = query(context, prompt)
    text += response
    time.sleep(0.5)

print(text)

100%|██████████| 13/13 [00:16<00:00,  1.24s/it]

1111110000000


In [47]:
text

'1111110000000'

# Person Analysis

In [32]:
response_files = glob.glob("./persons_predictions/*.txt")
all_sample_preds = []

for i in range(len(response_files)):
    response_file = f"./persons_predictions/sample_{i}.txt"
    with open(response_file, "r") as f:
        preds = f.read().split('\n')
        preds = preds[:-1]

    first = mode([i[0] for i in preds])
    second = mode([i[1] for i in preds])
    third = mode([i[2] for i in preds])
    
    all_sample_preds.append(first+second+third)
print(all_sample_preds)

['100', '100', '100', '100', '001', '001', '110', '100', '110', '100', '010', '010', '001', '010', '100', '010', '010', '110', '001', '001', '011', '001', '001', '001', '101', '001', '001', '001']


In [57]:
first_preds = [int(i[0]) for i in all_sample_preds]
second_preds = [int(i[1]) for i in all_sample_preds]
third_preds = [int(i[2]) for i in all_sample_preds]

In [58]:
with open("./persons_label.txt", "r") as f:
    gt = f.read().split("\n")
    print(gt)
    
first_gt = [int(i[0]) for i in gt]
second_gt = [int(i[1]) for i in gt]
third_gt = [int(i[2]) for i in gt]

['100', '100', '100', '100', '101', '101', '101', '100', '100', '100', '010', '010', '010', '010', '010', '010', '010', '110', '001', '001', '101', '001', '001', '001', '001', '001', '001', '001']


In [59]:
confusion_matrix(first_preds, first_gt)

array([[14,  3],
       [ 2,  9]])

In [61]:
confusion_matrix(second_preds, second_gt)

array([[17,  2],
       [ 3,  6]])

In [62]:
confusion_matrix(third_preds, third_gt)

array([[14,  1],
       [ 1, 12]])

In [54]:
tn, fp, fn, tp = confusion_matrix(third_preds, third_gt).ravel()

In [56]:
tp/(tp+fn)

0.9230769230769231

# Entire Dataset

In [68]:
oak = os.getenv("OAK")
clean_comments_folder = f"{oak}/samori/tiktok/comments/cleaned"

cleaned_df = pd.read_csv(f"{clean_comments_folder}/comments_3_cleaned.csv")
texts = cleaned_df['text']

In [78]:
context = """You are an AI assistant that helps to annotate social media comment data related to opioids and/or drug addiction, and you know everything about identifying the grammartical person (ie first, second, or third person) of a sentence or post"""

In [79]:
prompt_template = """I will give you an opioid-related social media comment to annotate. You will return a onehot integer of length 3 (eg. 010). If the comment contains a first-person or personal account with opioids put 1 in position 1 of the onehot integer else 0. If the comment contains a second-person experience with opioid, put 1 in position 2 of the onehot integer else 0. If the comment contains a third-person experience with opioid, put 1 in position 3 of the onehot integer else 0. Examples: return 100 for 'I am recovering from OUD', 010 for 'I am sorry to hear you lost your son to fentanyl OD', 001 for 'They take fent laced substances'. Note that some comments can have multiple persons-perspective and you will put 1 in the positions corresponding to the persons-perspective present. Example return 110 for 'I lost a partner. I am sorry to hear you lost someone too. My condolences to you and your family', and 011 for 'You should take lower dose. They should be careful with supplies from the street'. 
Annotate the following (only return the annotation):\n"""

In [86]:
annotations = ""
for i in tqdm(range(9)):
    text = texts[i]
    print(text)
    prompt = prompt_template + text
    response = query(context, prompt)
    annotations += response + "\n"
    time.sleep(0.5)

  0%|          | 0/9 [00:00<?, ?it/s]

many of them have no idea they are using fentanyl!


 11%|█         | 1/9 [00:01<00:09,  1.14s/it]

your children are spiritual warefare!


 22%|██▏       | 2/9 [00:02<00:08,  1.16s/it]

...

 78%|███████▊  | 7/9 [00:08<00:02,  1.18s/it]

Jackson. I know you put all you had every bit, into loving your boy


 89%|████████▉ | 8/9 [00:09<00:01,  1.17s/it]

His message to you.how proud of you🥺im so sorry. My babygirl was born 1/22/2000 departed 11/29/2020 I only wait for the day I hv her in my arms😭agai


100%|██████████| 9/9 [00:10<00:00,  1.16s/it]


In [87]:
print(annotations)

001
000
001
001
010
110
001
010
011



In [82]:
texts[1]

'your children are spiritual warefare!'